In [ ]:
import csv
import pywt
import pyedflib
import numpy as np
from spectrum import *
from os import listdir
from nitime import utils
import scipy.stats as sp
from os.path import isfile, join
from matplotlib import pyplot as plt
import pandas as pd
from scipy.signal import argrelextrema
import tqdm


from nitime.viz import plot_tseries
from nitime import algorithms as alg
from nitime.timeseries import TimeSeries
from tqdm import tqdm as tqdm

In [ ]:
!pip install spectrum
!pip install pyedflib
!pip install utils
!pip install nitime
!pip install tqdm

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


In [ ]:
#from google.colab import drive

# This will prompt for authorization.
#drive.mount('/content/drive') #remove the hash when running code first time

#import pandas as pd
import pickle
DATA_PATH = "Your data path"

In [ ]:
import pickle

In [ ]:
import glob
allFiles = glob.glob(DATA_PATH + "/*.dat")

In [ ]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [ ]:
import glob
allFiles = glob.glob(DATA_PATH + "/*.dat")
s_music = []
x=[]
for file_ in allFiles:
    df = open(file_,'rb')
    s_music.append(df)
for i in range(0,32):
  x.append(i)
  x[i]= pickle.load(s_music[i])
sun = []
moon = []
for i in range(0,32) :
  for j in range(0, 40):
    sun.append(pd.DataFrame(x[i]['data'][j]))
    moon.append(pd.DataFrame(x[i]['labels'][j]).T)
    #s_music_label.append(pd.DataFrame(x[i]['labels'][j]).T)



In [ ]:
def coeff_var(a):
    b = a #Extracting the data from the 32 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for i in b:
        mean_i = np.mean(i) #Saving the mean of array i
        std_i = np.std(i) #Saving the standard deviation of array i
        output[k] = std_i/mean_i #computing coefficient of variation
        k=k+1
    return np.sum(output)/32

In [ ]:
import heapq
from scipy.signal import argrelextrema


def first_diff(i):
    b=i    
    
    out = np.zeros(len(b))
    
    for j in range(len(i)):
        out[j] = b[j-1]-b[j]# Obtaining the 1st Diffs
        
        j=j+1
        c=out[1:len(out)]
    return c #returns first diff


def slope_mean(p):
    b = p #Extracting the data from the 32 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for i in b:
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]
        t_max = argrelextrema(x, np.greater)[0]
        amp_min = i[argrelextrema(x, np.less)[0]]
        t_min = argrelextrema(x, np.less)[0]
        t = np.concatenate((t_max,t_min),axis=0)
        t.sort()#sort on the basis of time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = first_diff(amp)
        
        t_diff = first_diff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q]         
        output[k] = np.mean(res) 
        k=k+1
    return np.sum(output)/32

In [ ]:
import heapq
from scipy.signal import argrelextrema


def first_diff(i):
    b=i    
    
    out = np.zeros(len(b))
    
    for j in range(len(i)):
        out[j] = b[j-1]-b[j]# Obtaining the 1st Diffs
        
        j=j+1
        c=out[1:len(out)]
    return c #returns first diff


def slope_var(p):
    b = p #Extracting the data from the 32 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for i in b:
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]#storing maxima value
        t_max = argrelextrema(x, np.greater)[0]#storing time for maxima
        amp_min = i[argrelextrema(x, np.less)[0]]#storing minima value
        t_min = argrelextrema(x, np.less)[0]#storing time for minima value
        t = np.concatenate((t_max,t_min),axis=0) #making a single matrix of all matrix
        t.sort() #sorting according to time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = first_diff(amp)
        
        t_diff = first_diff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q] #calculating slope        
    
        output[k] = np.var(res) 
        k=k+1#counting k
    return np.sum(output)/32

In [ ]:
def kurtosis(a):
    b = a # Extracting the data from the 32 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 32)
    k = 0; # For counting the current row no.
    for i in b:
        mean_i = np.mean(i) # Saving the mean of array i
        std_i = np.std(i) # Saving the standard deviation of array i
        t = 0.0
        for j in i:
            t += (pow((j-mean_i)/std_i,4)-3)
        kurtosis_i = t/len(i) # Formula: (1/N)*(summation(x_i-mean)/standard_deviation)^4-3
        output[k] = kurtosis_i # Saving the kurtosis in the array created
        k +=1 # Updating the current row no.
    return np.sum(output)/32

In [ ]:
def secDiffMean(a):
    b = a # Extracting the data of the 32 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 32)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    for i in b:
        t = 0.0
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
        for j in range(len(i)-2):
            t += abs(temp1[j+1]-temp1[j]) # Summing the 2nd Diffs
        output[k] = t/(len(i)-2) # Calculating the mean of the 2nd Diffs
        k +=1 # Updating the current row no.
    return np.sum(output)/32

In [ ]:
def secDiffMax(a):
    b = a # Extracting the data from the 32 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 32)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    t = 0.0
    for i in b:
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
        t = temp1[1] - temp1[0]
        for j in range(len(i)-2):
            if abs(temp1[j+1]-temp1[j]) > t :
                t = temp1[j+1]-temp1[j] # Comparing current Diff with the last updated Diff Max

        output[k] = t # Storing the 2nd Diff Max for channel k
        k +=1 # Updating the current row no.
    return np.sum(output)/32

In [ ]:
def skewness(arr):
    data = arr 
    skew_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0; #current cell position in the output array
   
    for i in data:
        skew_array[index]=sp.stats.skew(i,axis=0,bias=True)
        index+=1 #updating the cell position
    return np.sum(skew_array)/32

In [ ]:

def first_diff_mean(arr):
    data = arr 
    diff_mean_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0; #current cell position in the output array
   
    for i in data:
        sum=0.0#initializing the sum at the start of each iteration
        for j in range(len(i)-1):
            sum += abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
           
        diff_mean_array[index]=sum/(len(i)-1)
        index+=1 #updating the cell position
    return np.sum(diff_mean_array)/32

In [ ]:
def first_diff_max(arr):
    data = arr 
    diff_max_array = np.zeros(len(data)) #Initialinling the array as all 0s
    first_diff = np.zeros(len(data[0])-1)#Initialinling the array as all 0s 
    index = 0; #current cell position in the output array
   
    for i in data:
        max=0.0#initializing at the start of each iteration
        for j in range(len(i)-1):
            first_diff[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
            if first_diff[j]>max: 
                max=first_diff[j] # finding the maximum of the first differences
        diff_max_array[index]=max
        index+=1 #updating the cell position
    return np.sum(diff_max_array)/32

In [ ]:
def fractal_dimension(Z, threshold=0.9):

    # Only for 2d image
    assert(len(Z.shape) == 2)

    # From https://github.com/rougier/numpy-100 (#87)
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)

        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])


    # Transform Z into a binary array
    Z = (Z < threshold)

    # Minimal dimension of image
    p = min(Z.shape)

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)

    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))

    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

In [ ]:
#autogressiveModelParametersBurg

def autogressiveModelParametersBurg(labels):
    feature = []
    a1=[]
    a2=[]
    a3=[]
    model_order = 3
    for i in range(32):
        AR, rho, ref = arburg(labels[i], model_order)
        feature.append(AR);
    for i in range(32):
        a1.append(feature[i][0])
        a2.append(feature[i][1])
        a3.append(feature[i][2])
        
    
    ar1=np.mean(a1).real
    ar2=np.mean(a2).real 
    ar3=np.mean(a3).real
    
    br1=np.min(a1).real
    br2=np.min(a2).real 
    br3=np.min(a3).real
    
    cr1=np.max(a1).real
    cr2=np.max(a2).real 
    cr3=np.max(a3).real
    
    
    
    return ar1,ar2,ar3,br1,br2,br3,cr1,cr2,cr3

In [ ]:
#hjorth(3 feature)

def hjorth(input):                                             # function for hjorth 
    realinput = input
    hjorth_activity = np.zeros(len(realinput))
    hjorth_mobility = np.zeros(len(realinput))
    hjorth_diffmobility = np.zeros(len(realinput))
    hjorth_complexity = np.zeros(len(realinput))
    diff_input = np.diff(realinput)
    diff_diffinput = np.diff(diff_input)
    k = 0
    for j in realinput:
        hjorth_activity[k] = np.var(j)
        hjorth_mobility[k] = np.sqrt(np.var(diff_input[k])/hjorth_activity[k])
        hjorth_diffmobility[k] = np.sqrt(np.var(diff_diffinput[k])/np.var(diff_input[k]))
        hjorth_complexity[k] = hjorth_diffmobility[k]/hjorth_mobility[k]
        k = k+1
    return np.sum(hjorth_activity)/32, np.sum(hjorth_mobility)/32, np.sum(hjorth_complexity)/32                       #returning hjorth activity, hjorth mobility , hjorth complexity

In [ ]:
#maxPwelch

from scipy import signal

def maxPwelch(data_win,Fs):
 
    
    BandF = [0.1, 3, 7, 12, 30]
    PMax = np.zeros([32,(len(BandF)-1)]);
    
    for j in range(32):
        f,Psd = signal.welch(data_win[j,:], Fs)
        
        for i in range(len(BandF)-1):
            fr = np.where((f>BandF[i]) & (f<=BandF[i+1]))
            PMax[j,i] = np.max(Psd[fr])
    
    return np.sum(PMax[:,0])/32,np.sum(PMax[:,1])/32,np.sum(PMax[:,2])/32,np.sum(PMax[:,3])/32

In [ ]:
#wavelet_features

import pywt
def wavelet_features(epoch,channels):
    cA_values = []
    cD_values = []
    cA_mean = []
    cA_std = []
    cA_Energy =[]
    cD_mean = []
    cD_std = []
    cD_Energy = []
    Entropy_D = []
    Entropy_A = []
    wfeatures = []
    for i in range(channels):
        cA,cD=pywt.dwt(epoch[i,:],'coif1')
        cA_values.append(cA)
        cD_values.append(cD)		#calculating the coefficients of wavelet transform.
    for x in range(channels):   
        cA_mean.append(np.mean(cA_values[x]))
        #wfeatures.append(np.mean(cA_values[x]))
        
        cA_std.append(abs(np.std(cA_values[x])))
        #wfeatures.append(abs(np.std(cA_values[x])))
        
        cA_Energy.append(abs(np.sum(np.square(cA_values[x]))))
        #wfeatures.append(abs(np.sum(np.square(cA_values[x]))))
        
        cD_mean.append(np.mean(cD_values[x]))		# mean and standard deviation values of coefficents of each channel is stored .
        #wfeatures.append(np.mean(cD_values[x]))

        cD_std.append(abs(np.std(cD_values[x])))
        #wfeatures.append(abs(np.std(cD_values[x])))
        
        cD_Energy.append(abs(np.sum(np.square(cD_values[x]))))
        #wfeatures.append(abs(np.sum(np.square(cD_values[x]))))
        
        Entropy_D.append(abs(np.sum(np.square(cD_values[x]) * np.log(np.square(cD_values[x])))))
        #wfeatures.append(abs(np.sum(np.square(cD_values[x]) * np.log(np.square(cD_values[x])))))
        
        Entropy_A.append(abs(np.sum(np.square(cA_values[x]) * np.log(np.square(cA_values[x]))))) 
        #wfeatures.append(abs(np.sum(np.square(cA_values[x]) * np.log(np.square(cA_values[x])))))
        
    a1=np.mean(cA_mean)
    b1=np.mean(cA_std) 
    c1=np.mean(cA_Energy)  
    d1=np.mean(cD_mean)
    e1=np.mean(cD_std)  
    f1=np.mean(cD_Energy)
    g1=np.mean(Entropy_D)
    h1=np.mean(Entropy_A)
    
    a2=np.min(cA_mean)
    b2=np.min(cA_std) 
    c2=np.min(cA_Energy)  
    d2=np.min(cD_mean)
    e2=np.min(cD_std)  
    f2=np.min(cD_Energy)
    g2=np.min(Entropy_D)
    h2=np.min(Entropy_A)
    
    a3=np.max(cA_mean)
    b3=np.max(cA_std) 
    c3=np.max(cA_Energy)  
    d3=np.max(cD_mean)
    e3=np.max(cD_std)  
    f3=np.max(cD_Energy)
    g3=np.max(Entropy_D)
    h3=np.max(Entropy_A)
  
    
    return a1,a2,a3,b1,b2,b3,c1,c2,c3,d1,d2,d3,e1,e2,e3,f1,f2,f3,g1,g2,g3,h1,h2,h3

In [ ]:
final = pd.DataFrame()
for k in range(1280):
  rows1=sun[k]
  sigbufs=rows1.values
  sigbufs.shape
  df = pd.DataFrame(columns=['coeff_var'])


  #coeff_var-------------------------------------
  feature=[]
  for i in tqdm(np.arange(0,63,3)):
      epoch = sigbufs[0:32,i*128:(i+3)*128]
      feature.append(coeff_var(epoch))  
  df['coeff_var'] = feature 

  #fractal_dimension-------------------------------------
  feature=[]
  for i in tqdm(np.arange(0,63,3)):
      epoch = sigbufs[0:32,i*128:(i+3)*128]
      feature.append(fractal_dimension(epoch))  
  df['fractal_dimension'] = feature 


  #kurtosis--------------------------------------
  feature=[]
  for i in tqdm(np.arange(0,63,3)):
      epoch = sigbufs[0:32,i*128:(i+3)*128]
      feature.append(kurtosis(epoch))
  df['kurtosis'] = feature


  #secDiffMean-----------------------------------
  feature=[]
  for i in tqdm(np.arange(0,63,3)):
      epoch = sigbufs[0:32,i*128:(i+3)*128]
      feature.append(secDiffMean(epoch))
  df['secDiffMean'] = feature


  #secDiffMax------------------------------------
  feature=[]
  for i in tqdm(np.arange(0,63,3)):
      epoch = sigbufs[0:32,i*128:(i+3)*128]
      feature.append(secDiffMax(epoch))
  df['secDiffMax'] = feature


  #skewness--------------------------------------
  feature=[]
  for i in tqdm(np.arange(0,63,3)):
      epoch = sigbufs[0:32,i*128:(i+3)*128]
      feature.append(skewness(epoch))
  df['skewness'] = feature


  #first_diff_mean------------------------------
  feature=[]
  for i in tqdm(np.arange(0,63,3)):
      epoch = sigbufs[0:32,i*128:(i+3)*128]
      feature.append(first_diff_mean(epoch))
  df['first_diff_mean'] = feature


  #first_diff_max-------------------------------
  feature=[]
  for i in tqdm(np.arange(0,63,3)):
      epoch = sigbufs[0:32,i*128:(i+3)*128]
      feature.append(first_diff_max(epoch))
  df['first_diff_max'] = feature


  #autogressiveModelParametersBurg--------------
  a1=[]
  a2=[]
  a3=[]
  b1=[]
  b2=[]
  b3=[]
  c1=[]
  c2=[]
  c3=[]

  for i in tqdm(np.arange(0,63,3)):
    epoch = sigbufs[0:32:,i*128:(i+3)*128]
    aa1,aa2,aa3,bb1,bb2,bb3,cc1,cc2,cc3=autogressiveModelParametersBurg(epoch)
    a1.append(aa1)
    b1.append(bb1)
    c1.append(cc1)
    a2.append(aa2)
    b2.append(bb2)
    c2.append(cc2)
    a3.append(aa3)
    b3.append(bb3)
    c3.append(cc3)

  df['AR1_mean']=a1
  df['AR1_min']=b1
  df['AR1_max']=c1

  df['AR2_mean']=a2
  df['AR2_min']=b2
  df['AR2_max']=c2

  df['AR3_mean']=a3
  df['AR3_min']=b3
  df['AR3_max']=c3


  #hjorth----------------------------------------------------------
  a1=[]
  b1=[]
  c1=[]

  for i in tqdm(np.arange(0,63,3)):
    epoch = sigbufs[0:32:,i*128:(i+3)*128]
    a,b,c=hjorth(epoch)
    a1.append(a)
    b1.append(b)
    c1.append(c)
  df['hjorth_activity']=a1
  df['hjorth_mobility']=b1
  df['hjorth_complexity']=c1


  #maxPwelch-----------------------------------------------
  a1=[]
  b1=[]
  c1=[]
  d1=[]
  e1=[]
  f1=[]
  g1=[]
  h1=[]

  for i in tqdm(np.arange(0,63,3)):
    epoch = sigbufs[0:32:,i*128:(i+3)*128]
    a,b,c,d=maxPwelch(epoch,128)
    e=a/b
    f=a/c
    g=b/d
    h=(a+b)/(c)
    a1.append(a)
    b1.append(b)
    c1.append(c)
    d1.append(d)
    e1.append(e)
    f1.append(f)
    g1.append(g)
    h1.append(h)


  df['PMax1']=a1
  df['PMax2']=b1
  df['PMax3']=c1
  df['PMax4']=d1
  df['PRatio1']=e1
  df['PRatio2']=f1
  df['PRatio3']=g1
  df['PRatio4']=h1


  #wavelet_features-------------------------------------------
  a1=[]
  a2=[]
  a3=[]
  b1=[]
  b2=[]
  b3=[]
  c1=[]
  c2=[]
  c3=[]
  d1=[]
  d2=[]
  d3=[]
  e1=[]
  e2=[]
  e3=[]
  f1=[]
  f2=[]
  f3=[]
  g1=[]
  g2=[]
  g3=[]
  h1=[]
  h2=[]
  h3=[]

  for i in tqdm(np.arange(0,63,3)):
    epoch = sigbufs[0:32,i*128:(i+3)*128]
    aa1,aa2,aa3,bb1,bb2,bb3,cc1,cc2,cc3,dd1,dd2,dd3,ee1,ee2,ee3,ff1,ff2,ff3,gg1,gg2,gg3,hh1,hh2,hh3=wavelet_features(epoch,32)
    a1.append(aa1)
    b1.append(bb1)
    c1.append(cc1)
    d1.append(dd1)
    e1.append(ee1)
    f1.append(ff1)
    g1.append(gg1)
    h1.append(hh1)
    a2.append(aa2)
    b2.append(bb2)
    c2.append(cc2)
    d2.append(dd2)
    e2.append(ee2)
    f2.append(ff2)
    g2.append(gg2)
    h2.append(hh2)
    a3.append(aa3)
    b3.append(bb3)
    c3.append(cc3)
    d3.append(dd3)
    e3.append(ee3)
    f3.append(ff3)
    g3.append(gg3)
    h3.append(hh3)

  df['cA_mean_mean']=a1
  df['cA_mean_min']=a2
  df['cA_mean_max']=a3

  df['cA_std_mean']=b1
  df['cA_std_min']=b2
  df['cA_std_max']=b3

  df['cA_Energy_mean']=c1
  df['cA_Energy_min']=c2
  df['cA_Energy_max']=c3

  df['cD_mean_mean']=d1
  df['cD_mean_min']=d2
  df['cD_mean_max']=d3

  df['cD_std_mean']=e1
  df['cD_std_min']=e2
  df['cD_std_max']=e3

  df['cD_Energy_mean']=f1
  df['cD_Energy_min']=f2
  df['cD_Energy_max']=f3

  df['Entropy_D_mean']=g1
  df['Entropy_D_min']=g2
  df['Entropy_D_max']=g3

  df['Entropy_A_mean']=h1
  df['Entropy_A_min']=h2
  df['Entropy_A_max']=h3


  #target
  df['valence']=moon[k][0][0]
  df['arousal']=moon[k][1][0]
  df['dominance']=moon[k][2][0]
  df['liking']=moon[k][3][0]
  final = pd.concat([final,df])

In [ ]:
pd.DataFrame(final).to_csv('complete.csv')
from google.colab import files
files.download("complete.csv")

In [ ]:
final.shape()